In [ ]:
import time
import threading
import concurrent.futures
import json
import statistics
import random
import datetime
import pandas as pd
import numpy as np
from pydruid.db import connect
import json
import random
import os


conn = connect(host='localhost', port=8082, path='/druid/v2/sql/', scheme='http')
curs = conn.cursor()

#CREAZIONE DEL FILE PER RACCOGLIERE I RISULTATI
data = {}

for i in range(9):
    data[i] = {
        "Times": []
    }

filename ='../stats/reduced_execution_times_parallel.json'
file_exceptions='../stats/exceptions.json'

if not os.path.exists(filename):
    with open(filename, "w") as f:
        json.dump(data, f)
        print(f"Il file {filename} è stato creato con successo!")
else:
    print(f"Il file {filename} esiste già!")


with open(file_exceptions, 'w') as outfile:
    json.dump({}, outfile)



In [ ]:
ops=["<", ">", "="]
dimensions = {}
dimensions['princ']={}
dimensions['princ']['WARD'] = []
dimensions['princ']['DISTRICT'] = []
dimensions['sub']={}
dimensions['sub']['NEIGHBORHOOD_CLUSTER'] = []
dimensions['sub']['ANC'] = []
dimensions['sub']['BLOCK_GROUP'] = []
dimensions['sub']['VOTING_PRECINCT'] = []
dimensions['sub']['CENSUS_TRACT'] = []

def get_random_pc():
    index_ns = np.random.randint(0, 1)
    index_ew = np.random.randint(0, 1)
    ns = ["North", "South"]
    ew = ["East", "Weast"]
    return ns[index_ns], ew[index_ew]

def get_random_interval():
    timestamp_end = 1601159260
    timestamp_init = 631152000
    random_numbers = random.sample(range(timestamp_init, timestamp_end), 2)
    random_numbers.sort()
    date_init = datetime.datetime.fromtimestamp( random_numbers[0]).strftime('%Y-%m-%dT%H:%M:%S')
    date_end = datetime.datetime.fromtimestamp( random_numbers[1]).strftime('%Y-%m-%dT%H:%M:%S')
    return date_init, date_end

def get_random_sing_dimension(cat):
    return random.choice(list(dimensions[cat].keys()))

def get_random_mult_dimension(cat, n):
    indices = random.sample(list(dimensions[cat].keys()), n)
    return indices

def get_random_value(index1, index2):
    return random.choice(dimensions[index1][index2])

def get_random_year():
    return np.random.randint(1990, 2023)



In [ ]:
for key in list(dimensions.keys()):
    for sub_key in list(dimensions[key].keys()):
        query = 'SELECT DISTINCT("{dim}") FROM "crimesOtt"'.format(dim=sub_key)
        cur = curs.execute(query)
        res = [row[0] for row in cur.fetchall()]
        dimensions[key][sub_key] = res


In [ ]:
reduced_queries=[]
#SELECTION QUERY 
reduced_queries.append([0,
"""
SELECT SUM("count") AS "Count"
FROM "crimesOtt"
WHERE "NS" = '{ns}' AND "EW" = '{ew}' 
GROUP BY ()

"""
])

reduced_queries.append([1,"""
SELECT 
  "{dim_1}", 
  SUM("count") AS "total_crimes",
  CONCAT(ROUND(SUM(CAST("count" AS NUMERIC(10,2)))*100/ (SELECT SUM(CAST("count" AS NUMERIC(10,2))) FROM "crimesOtt"), 4), '%') AS "rate" 
FROM "crimesOtt"
WHERE "{dim_2}" = '{value}' AND "CRIMETYPE" = '{type}' AND EXTRACT(YEAR FROM FLOOR("__time" TO YEAR)) = {year} = '{year}'
GROUP BY 1
"""
])

reduced_queries.append([2,"""
SELECT
  APPROX_COUNT_DISTINCT_DS_THETA("theta_PSA") AS "approx_count_psa"
FROM "crimesOtt"
WHERE "{dim}" = '{value}' AND EXTRACT(YEAR FROM FLOOR("__time" TO YEAR)) = {year}
"""
])

reduced_queries.append([3,"""
SELECT
"year1",
"{dim_1}",
"max_count"
FROM(
  SELECT "year1", MAX("count1") AS "max_count"
    FROM(
      SELECT 
        EXTRACT(YEAR FROM FLOOR("__time" TO YEAR)) AS "year1",
        "{dim_1}",
        SUM("count") AS "count1"
      FROM "crimesOtt"
      GROUP BY 1, 2
    ) 
    GROUP BY 1
) 
LEFT JOIN 
(
  SELECT 
    EXTRACT(YEAR FROM FLOOR("__time" TO YEAR)) AS "year2",
    "{dim_1}",
    SUM("count") AS "count2"
  FROM "crimesOtt"
  GROUP BY 1, 2
)
ON "year1" = "year2" AND "max_count" = "count2"
"""
])

reduced_queries.append([4,"""
WITH
crimes_dim1 AS (
SELECT
EXTRACT(
  YEAR FROM FLOOR("__time" TO YEAR)) AS "year",
  "{dim1}",
  SUM("count") AS "count"
  FROM "crimesOtt"
  GROUP BY 1, 2
),
crimes_dim2 AS (
SELECT
EXTRACT(
  YEAR FROM FLOOR("__time" TO YEAR)) AS "year",
  "{dim2}",
  SUM("count") AS "count"
  FROM "crimesOtt"
  GROUP BY 1, 2
)

SELECT
t1."year",
t1."{dim1}",
t1."max_dim1_count",
t2."{dim2}",
t2."max_dim2_count"
FROM
(
  SELECT
  a."year",
  a."{dim1}",
  a1."max_dim1_count"
  FROM crimes_dim1 a
  INNER JOIN (
  SELECT
  "year",
  MAX("count") AS "max_dim1_count"
  FROM crimes_dim1
  GROUP BY 1
  ) a1
  ON a."year" = a1."year" AND a."count" = a1."max_dim1_count"
) AS t1
LEFT JOIN(
  SELECT
  b."year",
  b."{dim2}",
  b1."max_dim2_count"
  FROM crimes_dim2 b
  INNER JOIN (
  SELECT
  "year",
  MAX("count") AS "max_dim2_count"
  FROM crimes_dim2
  GROUP BY 1
  ) b1
  ON b."year" = b1."year" AND b."count" =  b1."max_dim2_count"
) AS t2
ON t1."year" = t2."year"
"""
])

reduced_queries.append([5,"""
SELECT
  APPROX_COUNT_DISTINCT_DS_THETA("theta_BLOCK") AS "approx_count_blocks"
FROM "crimesOtt"
WHERE "ANC" = '{value_A}' AND "NEIGHBORHOOD_CLUSTER" = '{value_NC}' AND "VOTING_PRECINCT" = '{value_VP}' AND "BLOCK_GROUP" = '{value_BG}' AND "CENSUS_TRACT" = '{value_CT}' AND "CRIMETYPE" = '{value_C}' AND EXTRACT(YEAR FROM FLOOR("__time" TO YEAR)) = {year}
"""
])

reduced_queries.append([6,"""
SELECT
  APPROX_COUNT_DISTINCT_DS_THETA("theta_OFFENSE_TYPE") AS "different_offense_type"
FROM "crimesOtt"
WHERE "DISTRICT"={value_dist} AND "{dim}" = '{value_dim}' 
"""
])

reduced_queries.append([7,"""
SELECT 
  EXTRACT(MONTH FROM FLOOR("__time" TO MONTH)) AS "month",
  ROUND(SUM(CAST("count" AS NUMERIC(10,2)))*100/(SELECT COUNT(DISTINCT(EXTRACT(YEAR FROM FLOOR("__time" TO YEAR)))) FROM "crimesOtt"))
FROM "crimesOtt"
WHERE "{dim}" = '{value}' AND "CRIMETYPE" = '{type}'
GROUP BY 1
"""])

reduced_queries.append([8, """
SELECT COUNT(*) 
FROM "crimesOtt"
WHERE "__time" >= '{date_init}' AND "__time" < '{date_end}' AND '{dim}' = '{value}'
"""])



In [ ]:
def build_query(index):
    query = reduced_queries[index]
    n_query = query[0]
    ex_query = query[1]

    if n_query == 0:
        ns_, ew_ = get_random_pc()
        return ex_query.format(ns=ns_, ew=ew_)
    elif n_query == 1:
        dim1_= get_random_sing_dimension("sub")
        dim2_= get_random_sing_dimension("princ")
        value_= get_random_value("princ", dim2_)
        crimetype  = random.choice(["Violent", "Non-Violent"])
        year_ = get_random_year()
        return ex_query.format(dim_1 = dim1_ , dim_2 = dim2_ , value = value_, type=crimetype, year=year_)
    elif n_query == 2:
        cat = random.choice(["sub", "princ"])
        dim_ = get_random_sing_dimension(cat) 
        value_ = get_random_value(cat, dim_)
        year_ = get_random_year()
        return ex_query.format(dim=dim_, value=value_, year=year_)
    elif n_query == 3:
        cat = random.choice(["sub", "princ"])
        dim_ = get_random_sing_dimension(cat)
        return ex_query.format(dim_1=dim_)
    elif n_query == 4:
        cat = random.choice(["sub", "princ"])
        dims = get_random_mult_dimension(cat, 2)
        return ex_query.format(dim1=dims[0], dim2=dims[1])
    elif n_query == 5:
        value_A_ = get_random_value("sub", "ANC") 
        value_NC_ = get_random_value("sub", "NEIGHBORHOOD_CLUSTER")
        value_VP_ = get_random_value("sub", "VOTING_PRECINCT")
        value_BG_ = get_random_value("sub", "BLOCK_GROUP")
        value_CT_ = get_random_value("sub", "CENSUS_TRACT")
        value_C_ = np.random.choice(["Violent","Non-Violent"])
        year_ = get_random_year()
        return ex_query.format(value_A=value_A_ , value_NC=value_NC_ , value_VP=value_VP_ , value_BG=value_BG_ , value_CT=value_CT_ , value_C=value_C_ , year=year_)
    
    elif n_query == 6:
        cat = random.choice(["sub", "princ"])
        dim_ = get_random_sing_dimension(cat)
        value_ = get_random_value(cat, dim_)
        value_dist_ = get_random_value("princ", "DISTRICT")
        year_ = get_random_year()
        return ex_query.format(value_dist=value_dist_, dim=dim_, value_dim=value_, year=year_)
    elif n_query == 7:
        dim_=get_random_sing_dimension("princ")
        value_=get_random_value("princ", dim_)
        type_=np.random.choice(["Violent","Non-Violent"])
        return ex_query.format(dim=dim_, value=value_, type=type_)
    elif n_query == 8:
        date_init_, date_end_ = get_random_interval()
        cat = random.choice(["sub", "princ"])
        dim_ = get_random_sing_dimension(cat)
        value_ = get_random_value(cat, dim_)
        return ex_query.format(date_init = date_init_, date_end = date_end_, dim=dim_, value=value_)
    


In [ ]:
#FUNZIONI PER ESEGUIRE QUERY
def update_times(key, new_time, file):
    with open(file, 'r+') as file:
        data = json.load(file)
        data[key]["Times"].append(new_time)
        file.seek(0)
        json.dump(data, file, indent=4)
        file.truncate()

def update_metrics(key, filename):
    with open(filename, 'r+') as file:
        data = json.load(file)
        execution_times = data[key]["Times"]
        mean = statistics.mean(execution_times)
        stdev = statistics.stdev(execution_times)
        data[key]["Mean"] = mean
        data[key]["Stdev"] = stdev
        file.seek(0)
        json.dump(data, file, indent=4)
        file.truncate()

def execute_query(query):
    start_time = time.time()
    curs.execute(query)
    end_time = time.time()
    return (end_time-start_time)

def execute_query_parallel(query, data):
    start_time = time.time()
    curs.execute(query[1])
    end_time = time.time()
    data[str(query[0])]["Times"].append(end_time-start_time)
    

def get_times(file):
    with open(file, 'r+') as file:
        return json.load(file)

def update_times(file, data):
    with open(file, 'r+') as file:
        file.seek(0)
        json.dump(data, file, indent=4)
        file.truncate()



In [ ]:
dim_queries = 30000
cycle = 3000
list_time_sleep = [0.1, 0,2, 0.3]
list_num_threads = [9, 18, 27]
query_red=[]
for i in range(0, dim_queries):
    random.shuffle(reduced_queries)
    for query in reduced_queries:
        fquery = build_query(query[0])
        query_red.append([query[0], fquery])
random.shuffle(query_red)
data = get_times(filename)
with open("file.txt", "a") as file:
    for i in range(0, cycle):
        with concurrent.futures.ThreadPoolExecutor(max_workers=num_threads) as executor:
            time_sleep=random.choice(list_time_sleep)
            num_threads=random.choice(list_num_threads)
            try:
                time.sleep(time_sleep)
                executor.submit(execute_query_parallel, query_red[i], data)
                
            except Exception as e:
                with open(file_exceptions, 'r+') as file:
                    data = json.load(file)
                    error = {}
                    error["datasource"]="reduced"
                    error["e"]=str(e)
                    error["id"]=query_red[i][0]
                    error["query"]=query_red[i][1]
                    data[i] =error
                    file.seek(0)
                    json.dump(data, file, indent=1)
                    file.truncate()          
update_times(filename, data)
